**Troubleshooting Scripts - Memory Usage**

Dmitri V. Korotkevitch (MCM, MVP)

email: [dk@aboutsqlserver.com](mailto:dk@aboutsqlserver.com)      blog: [https://aboutsqlserver.com](https://aboutsqlserver.com/) code: [https://github.com/aboutsqlserver/code](https://github.com/aboutsqlserver/code)

SQL Server Advanced Troubleshooting and Performance Tuning (O'Reilly, 2022)      ISBN: 978-1098101923

**Checking Minimum and Maximum Server Memory Settings**

In [ ]:
SELECT *
FROM sys.configurations WITH (NOLOCK)
WHERE name IN (N'min server memory (MB)', 'max server memory (MB)')
OPTION (MAXDOP 1, RECOMPILE);

**OS Memory Usage**

In [ ]:
SELECT 
    total_physical_memory_kb / 1024 AS [Physical Memory (MB)] 
    ,available_physical_memory_kb / 1024 AS [Available Memory (MB)] 
    ,total_page_file_kb / 1024 AS [Page File Commit Limit (MB)]
    ,available_page_file_kb / 1024 AS [Available Page File (MB)] 
    ,(total_page_file_kb - total_physical_memory_kb) / 1024 
        AS [Physical Page File Size (MB)]
    ,system_cache_kb / 1024 AS [System Cache (MB)]
    /* Values: LOW/HIGH/STEADY */
    ,system_memory_state_desc AS [System Memory State] 
FROM sys.dm_os_sys_memory WITH (NOLOCK)
OPTION (RECOMPILE, MAXDOP 1);

**SQL Server Memory Usage**

In [ ]:
SELECT 
    physical_memory_in_use_kb / 1024 
        AS [SQL Server Memory Usage (MB)]
    ,locked_page_allocations_kb / 1024 
        AS [SQL Server Locked Pages Allocation (MB)]
    ,large_page_allocations_kb / 1024 
        AS [SQL Server Large Pages Allocation (MB)] 
    ,memory_utilization_percentage
    ,available_commit_limit_kb
    ,process_physical_memory_low /* May indicate memory pressure */
    ,process_virtual_memory_low
FROM sys.dm_os_process_memory WITH (NOLOCK)
OPTION (RECOMPILE, MAXDOP 1);

**NUMA Memory Usage**

In [ ]:
SELECT 
	n.node_id
	,n.node_state_desc
	,n.memory_node_id
	,n.online_scheduler_count
	,CONVERT(DECIMAL(16,3), m.pages_kb /1024.) AS committed_memory_mb
	,CONVERT(DECIMAL(16,3), m.locked_page_allocations_kb / 1024.) AS locked_memory_mb 
	,CONVERT(DECIMAL(16,3), m.foreign_committed_kb / 1024.) AS foreign_memory_mb
	,CONVERT(DECIMAL(16,3), m.target_kb / 1024.) AS target_memory_mb -- SQL Server 2016+
FROM 
	sys.dm_os_nodes n WITH (NOLOCK)
		JOIN sys.dm_os_memory_nodes m WITH (NOLOCK) ON 
			n.memory_node_id = m.memory_node_id
WHERE 
	n.node_state_desc <> N'ONLINE DAC' 
OPTION (MAXDOP 1, RECOMPILE);

**Page Life Expectancy**

In [ ]:
SELECT object_name, counter_name, instance_name, cntr_value as [PLE(sec)]
FROM sys.dm_os_performance_counters WITH (NOLOCK) 
WHERE counter_name = 'Page life expectancy'
OPTION (MAXDOP 1,RECOMPILE);

**Database Buffer Pool Usage**

In [ ]:
;WITH BufPoolStats
AS
(
    SELECT 
        database_id
        ,COUNT_BIG(*) AS page_count
        ,CONVERT(DECIMAL(16,3),COUNT_BIG(*) * 8 / 1024.) AS size_mb
        ,AVG(read_microsec) AS avg_read_microsec
    FROM 
        sys.dm_os_buffer_descriptors WITH (NOLOCK)
    GROUP BY 
        database_id
)
SELECT 
    DB_NAME(database_id) AS [DB]
    ,size_mb
    ,page_count
    ,avg_read_microsec
    ,CONVERT(DECIMAL(5,2), 100. * (size_mb / SUM(size_mb) OVER())) 
        AS [Percent]
FROM 
    BufPoolStats
ORDER BY 
    size_mb DESC
OPTION (MAXDOP 1, RECOMPILE);

**Memory Clerk Memory Usage**

In [ ]:
SELECT TOP 15
    [type] AS [Memory Clerk]
    ,CONVERT(DECIMAL(16,3),SUM(pages_kb) / 1024.0) AS [Memory Usage(MB)]
FROM sys.dm_os_memory_clerks WITH (NOLOCK)
GROUP BY [type]
ORDER BY sum(pages_kb) DESC
OPTION (RECOMPILE, MAXDOP 1);

**Analyzing memory grants.**

Uncomment WHERE clause if you want to see pending requests only

In [ ]:
SELECT
    mg.session_id
    ,t.text AS [sql]
    ,qp.query_plan AS [plan]
    ,mg.is_small  /* Resource Semaphore Queue information */
    ,mg.dop
    ,mg.query_cost
    ,mg.request_time
    ,mg.grant_time
    ,mg.wait_time_ms
    ,mg.required_memory_kb
    ,mg.requested_memory_kb
    ,mg.granted_memory_kb
    ,mg.used_memory_kb
    ,mg.max_used_memory_kb
    ,mg.ideal_memory_kb
FROM
    sys.dm_exec_query_memory_grants mg WITH (NOLOCK)
        CROSS APPLY sys.dm_exec_sql_text(mg.sql_handle) t
        CROSS APPLY sys.dm_exec_query_plan(mg.plan_handle) qp
--WHERE 
--    mg.grant_time IS NULL
ORDER BY
    mg.requested_memory_kb DESC
OPTION (RECOMPILE, MAXDOP 1);
